<a href="https://colab.research.google.com/github/siddarthgowda/Audio_Analysis/blob/main/audio_transcript_py1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import whisper

model = whisper.load_model("base")  # Use "small" or "large" for better accuracy

def transcribe_audio(file_path):
    result = model.transcribe(file_path)
    return result["text"]

# Example usage
text = transcribe_audio("/content/sample_data/audio3.wav")
print("text:",text)


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


text:  Thank you for calling life support service. This is Jane. How can I help you? I have a question about something that I received in a mail. Sure. I'll be glad to assist you regarding that mail. Perfect. Can you please tell me more about the mail you received? Yeah, sure. Well, you see, couple of weeks ago. I met with an accident and I was rushed to hospital. My knees slammed into the dashboard. But I feel better now. I was under the impression that my insurance will cover the whole bill. But this paperwork says something else. I will be happy to check your account so that I can help you better. Okay, thank you. May I have your complete name, sir? My first name is John. Last name is Williams. And your member ID, John? My ID number is 1dg4t6nk62. And your zip code? 98101 Fifth Avenue.


In [5]:
from pyannote.audio import Pipeline

diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="HF_AUTH_KEY")

def get_speaker_segments(file_path):

    diarization = diarization_pipeline({"uri": "sample", "audio": file_path})
    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({"start": turn.start, "end": turn.end, "speaker": speaker})
    return segments

# Example usage
segments = get_speaker_segments("/content/sample_data/audio3.wav")


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


In [7]:
import whisper
from pyannote.audio import Pipeline
from pyannote.core import Segment

# Load Whisper model
model = whisper.load_model("base")  # Use "small" or "large" for better accuracy

# Load Pyannote speaker diarization model (requires Hugging Face token)
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="HF_AUTH_KEY"
)

def transcribe_audio(file_path):
    """Transcribes the audio file using Whisper."""
    result = model.transcribe(file_path)
    return result["segments"]  # Returns list of timestamped text segments

def get_speaker_segments(file_path):
    """Performs speaker diarization using Pyannote."""
    diarization = diarization_pipeline({"uri": "sample", "audio": file_path})
    segments = []

    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({
            "start": turn.start,
            "end": turn.end,
            "speaker": speaker
        })
    return segments

def align_speakers(transcript_segments, speaker_segments):
    """Matches transcribed text with speaker labels based on timestamps."""
    dialogue = []

    for segment in transcript_segments:
        text = segment["text"]
        start, end = segment["start"], segment["end"]

        # Find the matching speaker
        speaker = "Unknown"
        for s in speaker_segments:
            if s["start"] <= start <= s["end"]:
                speaker = s["speaker"]
                break

        dialogue.append(f"{speaker}: {text}")

    return "\n".join(dialogue)

# Main execution
if __name__ == "__main__":
    file_path = "/content/sample_data/audio3.wav"  # Update with your audio file

    print("Transcribing audio...")
    transcript_segments = transcribe_audio(file_path)

    print("Detecting speakers...")
    speaker_segments = get_speaker_segments(file_path)

    print("Aligning transcript with speakers...")
    conversation = align_speakers(transcript_segments, speaker_segments)

    print("\n--- Conversation ---")
    print(conversation)


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.
Transcribing audio...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting speakers...
Aligning transcript with speakers...

--- Conversation ---
Unknown:  Thank you for calling life support service.
Unknown:  This is Jane. How can I help you?
Unknown:  I have a question about something that I received in a mail.
Unknown:  Sure. I'll be glad to assist you regarding that mail.
Unknown:  Perfect.
Unknown:  Can you please tell me more about the mail you received?
SPEAKER_00:  Yeah, sure.
Unknown:  Well, you see, couple of weeks ago.
Unknown:  I met with an accident and I was rushed to hospital.
Unknown:  My knees slammed into the dashboard.
Unknown:  But I feel better now.
Unknown:  I was under the impression that my insurance will cover the whole bill.
Unknown:  But this paperwork says something else.
Unknown:  I will be happy to check your account so that I can help you better.
Unknown:  Okay, thank you.
Unknown:  May I have your complete name, sir?
Unknown:  My first name is John. Last name is Williams.
Unknown:  And your member ID, John?
Unknown:  

In [8]:
import google.generativeai as genai

# Set API key directly
api_key = "GEMINI_API_KEY"  # Replace with a valid API key

# Configure API key
genai.configure(api_key=api_key)

# Define transcript
transcript = """
Thank you for calling life support service. This is Jane. How can I help you?
I have a question about something that I received in a mail. Sure.
I'll be glad to assist you regarding that mail. Perfect. Can you please tell me more about the mail you received?
Yeah, sure. Well, you see, couple of weeks ago. I met with an accident and I was rushed to hospital. My knees slammed into the dashboard. But I feel better now. I was under the impression that my insurance will cover the whole bill. But this paperwork says something else.
I will be happy to check your account so that I can help you better.
Okay, thank you.
May I have your complete name, sir?
My first name is John. Last name is Williams.
And your member ID, John?
My ID number is 1dg4t6nk62.
And your zip code?
98101 Fifth Avenue.
"""

# Define prompt
prompt = f"""
Analyze the following call transcript and rate the following aspects on a scale of 1 to 10:
- Overall transcript analysis score
- User-Agent interaction score
- Agent communication score
- User response score
- User language clarity
- Solution given clarity

Also, provide a sentiment analysis (positive, neutral, negative) with a brief explanation.

Transcript:
{transcript}
"""

# Initialize the Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")

# Generate response
response = model.generate_content(prompt)

# Print analysis
print("Analysis Results:\n")
print(response.text)


Analysis Results:

Okay, here's the analysis of the call transcript:

**Ratings (1-10 scale):**

*   **Overall Transcript Analysis Score:** 7 - The transcript shows a reasonably smooth interaction, but lacks resolution. More information and follow-up are needed.
*   **Agent Interaction Score:** 8 - Jane is polite and offers assistance. She seeks necessary information to help the user. Her prompts are clear.
*   **User Response Score:** 7 - John provides the requested information, albeit with a slightly rambling explanation initially. He's cooperative.
*   **Agent Communication Clarity:** 9 - Jane's communication is clear, concise, and professional.
*   **User Language Clarity:** 7 - John's initial explanation is a bit verbose, but he ultimately provides the needed information.
*   **Solution Given Clarity:** 2 - There is no solution given in this snippet. The agent is only in the information gathering stage.

**Sentiment Analysis:**

*   **Sentiment:** Neutral
*   **Explanation:** The 

In [9]:
from google.colab import files
uploaded = files.upload()

Saving audio3.wav to audio3.wav
